In [1]:
# importing required modules
from zipfile import ZipFile     # to unzip files
import os                       # to add OS directories
from urllib import request      # to download web files
import csv                      # to read/write csv files
import shutil                   # to remove OS directories

In [13]:
# Defining data files to upload {file description, [file name, download link]}
data_files = {
    "sib_ativos": ["sib_ativos.zip", "https://dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_operadora/sib_ativos.zip"]
}

# Create Data directorie
if "data" not in os.listdir():
    os.mkdir("data")

def download_data_files(data_files):
    print("Downloading data files...")
    
    for file in data_files.keys():
        print("Downloading: ", file)
        # Define the local filename to save data
        local_file = "data/" + data_files[file][0]        

        # Define the remote file to retrieve
        remote_url = data_files[file][1]
        
        # Download remote and save locally
        request.urlretrieve(remote_url, local_file)
        print (file, "sucess !!")

def unzip_data_files(data_files):
    for file in data_files:
        
        # specifying the zip file name
        file_name = "data/" + data_files[file][0] 
        
        # creating unzip directorie
        dir_name = "data" + os.sep + file
        
        if dir_name in os.listdir():
            os.mkdir(dir_name)

        # specifyng file path
        file_path = file + os.sep

        # opening the zip file in READ mode
        with ZipFile(file_name, 'r') as zip:
            # printing all the contents of the zip file
            zip.printdir()
        
            # extracting all the files
            print('Extracting all the files now...')
            zip.extractall(file_path)
            print('Done!')

def create_sib_ativos_template(new_file_name):
    # specifyng file path
    dir_name = "data" + os.sep + "sib_ativos"

    # creating a new folder to saving modified csv files
    if 'sib_ativos' not in os.listdir("data" + os.sep):
        os.mkdir(dir_name)

    # Creating first line with headers from dataset
    cols = str("CD_OPERADORA;LG_BENEFICIARIO_ATIVO;DT_NASCIMENTO;TP_SEXO;DT_CONTRATACAO;\
ID_BENE_TIPO_DEPENDENTE;SG_UF;DT_CANCELAMENTO;CD_BENE_MOTIV_CANCELAMENTO")

    with open(new_file_name, 'w', newline='', encoding='cp1252') as csvfile:
        print(cols, file=csvfile)
        csvfile.close() 

def extract_sib_features(filepath, new_file_name):
      
    with open(filepath, newline='', encoding='cp1252') as csvfile:
        
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        
        # open file to write main data features
        csvfile = open(new_file_name, 'a', newline='', encoding='cp1252')

        for row in spamreader:
            row_values = str(row).split(";")
            # Checking if line is a header
            if row_values[1] != 'CD_OPERADORA':
                nrow = (row_values[1].replace('"', ''),
                        row_values[6].replace('"', ''), 
                        row_values[7].replace('"', ''),
                        row_values[8].replace('"', ''),
                        row_values[13].replace('"', ''),
                        row_values[14].replace('"', ''),
                        row_values[18].replace('"', ''),
                        row_values[23].replace('"', ''),
                        row_values[25].replace('"', '')
                    )

                spamwriter = csv.writer(csvfile, delimiter=';',
                                        quotechar=' ', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerow(nrow)
        
    csvfile.close()
    print(new_file_name, " done!!")

if __name__ =="__main__":
    download_data_files(data_files)
    unzip_data_files(data_files)
    
    new_file_name = "data" + os.sep + "sib_ativos" + os.sep + "all_sib.csv"
    create_sib_ativos_template(new_file_name)

    for file in os.listdir('sib_ativos'):
        print("Removing unnecessary features from: ", file)
        filepath = "sib_ativos" + os.sep + file
        #new_file_name = "data" + os.sep + "sib_ativos" + os.sep + file
        extract_sib_features(filepath, new_file_name)
        print("Unnecessary Features were removed!")
    
    shutil.rmtree("sib_ativos")

File Name                                             Modified             Size
sib_202404_AC.csv                              2024-06-05 04:16:54      8342161
sib_202404_AL.csv                              2024-06-05 04:16:54    105120722
sib_202404_AM.csv                              2024-06-05 04:16:56    148504486
sib_202404_AP.csv                              2024-06-05 04:16:58     13893684
sib_202404_BA.csv                              2024-06-05 04:17:02    452411419
sib_202404_CE.csv                              2024-06-05 04:17:08    352408899
sib_202404_DF.csv                              2024-06-05 04:17:12    208283763
sib_202404_ES.csv                              2024-06-05 04:17:16    257830588
sib_202404_GO.csv                              2024-06-05 04:17:20    290809685
sib_202404_MA.csv                              2024-06-05 04:17:22    115447323
sib_202404_MG.csv                              2024-06-05 04:17:36   1092103557
sib_202404_MS.csv                       

---
### Identificando a posição das features de interesse

In [7]:
# Colunas de interesse
cols = ['LG_BENEFICIARIO_ATIVO', 'DT_NASCIMENTO', 'TP_SEXO',    
       'DT_CONTRATACAO', 'ID_BENE_TIPO_DEPENDENTE', 'SG_UF',
       'LG_RESIDE_EXTERIOR', 'DT_CANCELAMENTO', 'CD_BENE_MOTIV_CANCELAMENTO']

# Todos os cabeçalhos do dataset
header = ["ID_TEMPO_COMPETENCIA", "CD_OPERADORA", "DT_INCLUSAO", "CD_BENE_MOTV_INCLUSAO", "IND_PORTABILIDADE", "ID_MOTIVO_MOVIMENTO", "LG_BENEFICIARIO_ATIVO", 
"DT_NASCIMENTO", "TP_SEXO", "CD_PLANO_RPS", "CD_PLANO_SCPA", "NR_PLANO_PORTABILIDADE", "DT_PRIMEIRA_CONTRATACAO", "DT_CONTRATACAO", "ID_BENE_TIPO_DEPENDENTE", 
"LG_COBERTURA_PARCIAL", "LG_ITEM_EXCLUIDO_COBERTURA", "CD_MUNICIPIO", "SG_UF", "LG_RESIDE_EXTERIOR", "DT_REATIVACAO", "DT_ULTIMA_REATIVACAO", 
"DT_ULTIMA_MUDA_CONTRATUAL", "DT_CANCELAMENTO", "DT_ULTIMO_CANCELAMENTO", "CD_BENE_MOTIV_CANCELAMENTO", "DT_CARGA"]

index = []

for i in cols:
       index.append(header.index(i))
print ("Os indices desejados são:", index)

# Testando a extração das features de interesse

text = '"202211";"334189";"2009-02";11;"NAO";74;1;"2006";"M";;;;"2008-09";"2008-09";1;0;0;"140010";"RR";0;;;;;;;"2023-01"'
lista = text.split(";")

nrow = (lista[6], lista[7], lista[8], lista[13], lista[14], lista[18], lista[19], lista[23], lista[25])
nrow

Os indices desejados são: [6, 7, 8, 13, 14, 18, 19, 23, 25]


('1', '"2006"', '"M"', '"2008-09"', '1', '"RR"', '0', '', '')

In [1]:
# checking all_sib.csv data
import pandas as pd

In [4]:
arquivo = "data" + os.sep + "sib_ativos" + os.sep + "all_sib.csv"

df = pd.read_csv(arquivo, sep=";", nrows=1000000)
df

<bound method NDFrame.head of         CD_OPERADORA  LG_BENEFICIARIO_ATIVO  DT_NASCIMENTO TP_SEXO  \
0             388092                      1         1987.0       F   
1             388092                      1         1984.0       M   
2             388092                      1         1951.0       F   
3             388092                      1         1964.0       M   
4             388092                      1         1966.0       F   
...              ...                    ...            ...     ...   
499995        311961                      1         1971.0       F   
499996        311961                      1         2012.0       M   
499997        311961                      1         1975.0       F   
499998        311961                      1         2011.0       M   
499999        311961                      1         2014.0       M   

       DT_CONTRATACAO  ID_BENE_TIPO_DEPENDENTE SG_UF DT_CANCELAMENTO  \
0             2005-03                      4.0    AM     